In [1]:
import cv2 as cv
import numpy as np

In [ ]:
# Resmi oku
#src = cv.imread("messi.png")
src = cv.imread("cat.png")

# Resmi yeniden boyutlandır
src = cv.resize(src, (0,0), fx=0.3, fy=0.3)

# ROI Seçimi
r   = cv.selectROI('input', src, False)

In [ ]:
# ROI Kırpma (ilgili bölgenin sadece o kısmını çıkarma)
roi = src[int(r[1]):int(r[1]+r[3]),
          int(r[0]):int(r[0]+r[2])]

In [ ]:
# Görüntüye dikdörgen çizme
img = src.copy()
cv.rectangle(img, 
             (int(r[0]),  int(r[1])),                            # Dikdörtgenin sol üst köşesi
             ((int(r[0]) + int(r[2])), (int(r[1]) + int(r[3]))), # Dikdörtgenin sağ alt köşesi
             (255, 0, 0), # Dikdörtgenin rengi (mavi)
             2);          # Çizgi kalınlığı

In [ ]:
# GrabCut algoritması için gerekli maskeler
mask = np.zeros(src.shape[:2], dtype=np.uint8) # Maske: başlangıçta siyah (0)
bgdmodel = np.zeros((1, 65), np.float64) # Arka plan modeli (boş dizi)
fgdmodel = np.zeros((1, 65), np.float64) # Ön plan modeli (boş dizi)

In [ ]:
# Seçilen bölgeyi arka plandan ayırmak için GrabCut algoritmasını kullanma
rect = (int(r[0]), int(r[1]), int(r[2]), int(r[3]))  # ROI'nin rect formatı: (x, y, width, height)
cv.grabCut(src, 
           mask,                        # Güncellenecek maske
           rect,                        # Dikdörtgen koordinatları
           bgdmodel,                    # Arka plan modeli
           fgdmodel,                    # Ön plan modeli
           25,                          # İterasyon sayısı
           mode=cv.GC_INIT_WITH_RECT);  # Dikdörtgen modu kullan

Not: iterasyon sayisi ile uğraşarak iyileştirmeler yapılabilir.

In [ ]:
# Maskeyi güncelle (1 ve 3 olan pikseller ön plandır, geri kalanlar arka plan)
mask2 = np.where((mask==1)+(mask==3), 
                 255, 
                 0
                 ).astype('uint8')

In [ ]:
background = cv.imread("volcano.png")

h, w, ch = src.shape
background = cv.resize(background, (w, h))
mask = np.zeros(src.shape[:2], dtype=np.uint8) # Maske: başlangıçta siyah (0)
bgdmodel = np.zeros((1, 65), np.float64) # Arka plan modeli (boş dizi)
fgdmodel = np.zeros((1, 65), np.float64) # Ön plan modeli (boş dizi)

In [ ]:
cv.grabCut(src, 
           mask,                        # Güncellenecek maske
           rect,                        # Dikdörtgen koordinatları
           bgdmodel,                    # Arka plan modeli
           fgdmodel,                    # Ön plan modeli
           5,                           # İterasyon sayısı
           mode=cv.GC_INIT_WITH_RECT);  # Dikdörtgen modu kullan

mask2 = np.where((mask==1)+(mask==3), 
                 255, 
                 0
                 ).astype('uint8')

In [ ]:
se = cv.getStructuringElement(cv.MORPH_RECT,
                              (3, 3))

cv.dilate(mask2,
          se,
          mask2)

mask2 = cv.GaussianBlur(mask2, 
                        (5,5), 
                        0)

In [ ]:
background = cv.GaussianBlur(background,
                             (0, 0),
                             15)
mask2 = mask2/255.0
a = mask2[..., None]
result = a* (src.astype(np.float32)) + (1 - a) * (background.astype(np.float32))

In [ ]:
# Sonucu bitwise_and ile görüntüye uygula (arka planı sil)
#result = cv.bitwise_and(src, 
 #                       src, 
  #                      mask=mask2)

In [ ]:
# Sonuçları göster
cv.imshow('roi', roi.astype(np.uint8))
cv.imshow('result', result.astype(np.uint8))
cv.waitKey(0)

27

In [ ]:
cv.destroyAllWindows()